In [1]:
from character import Character

In [70]:
class Rogue(Character):
    def __init__(self, *kwargs):
        super(Rogue, self).__init__(*kwargs)


class Knight(Character):
    def __init__(self, *kwargs):
        super(Knight, self).__init__(*kwargs)
        self.__reduce_dmg = (1 - 0.25)
        self.__reduce_atk = (1 - 0.2)

    def _get_caused_dmg(self, other):
        assert isinstance(other, Character)
        assert self is not other

        return max(1, int(round((self._lvl * 10) - other._lvl) * self.__reduce_atk))

    def _take_physical_damage(self, amount):
        assert isinstance(amount, int)
        assert amount >= 0
        self._health_cur = max(0, self._health_cur -
                               int(round(amount * self.__reduce_dmg)))


class Mage(Character):
    def __init__(self, *kwargs):
        super(Mage, self).__init__(*kwargs)
        self.__reduce_dmg = (1 - (-0.5))
        self.__reduce_atk = (1 - (-0.25))

    def attack(self, other):
        assert isinstance(other, Character)

        assert self is not other

        if not self.is_alive():
            raise Warning("Character is dead!")
        # 나중에 여기를 _take_magical_damage 로 변경하면..
        # 그래서 미리 Overriding 해둠
        other._take_physical_damage(self._get_caused_dmg(other))

    def _get_caused_dmg(self, other):
        assert isinstance(other, Character)
        assert self is not other

        return max(1, int(((self._lvl * 10) - other._lvl) * self.__reduce_atk))

    def _take_physical_damage(self, amount):
        assert isinstance(amount, int)
        assert amount >= 0
        self._health_cur = max(0, self._health_cur -
                               int(round(amount * self.__reduce_dmg)))

In [53]:
r = Rouge("hanyul", 12)
k = Knight("Arthur", 12)
print(k)
print(r)
r.attack(k)
k.attack(r)
print(k)
print(r)

Arthur (Knight, 12, 600/600)
hanyul (Rouge, 12, 600/600)
Arthur (Knight, 12, 519/600)
hanyul (Rouge, 12, 514/600)


In [51]:
r = Rouge("hanyul", 12)
k = Knight("Arthur", 12)
print(k)
print(r)
r.attack(k)
k.attack(r)
print(k)
print(r)

Arthur (Knight, 12, 600/600)
hanyul (Rouge, 12, 600/600)
Arthur (Knight, 12, 492/600)
hanyul (Rouge, 12, 492/600)


In [10]:
r = Rouge("hanyul", 12)
k = Knight("Arthur", 12)
k.attack(r)
print(k)
print(r)

Arthur (Knight, 12, 600/600)
hanyul (Rouge, 12, 492/600)


In [60]:
def attack(c1, c2):
    print("{} attacks {} ".format(c1, c2), end="")

    if not c2.is_alive():
        print("{} is beating a dead horse!".format(c1.get_name()))
        return

    life_before = c2.get_health()[0]
    c1.attack(c2)
    life_after = c2.get_health()[0]
    dmg = life_before-life_after

    print("and hits for {} damage. ".format(dmg), end="")

    if c2.is_alive():
        life = c2.get_health()
        perc = round(100 * life[0] / life[1], 1)
        print("{} still has {} ({}%) health".format(
            c2.get_name(), life[0], perc))
    else:
        print("{} died!".format(c2.get_name()))


k = Knight("Arthur", 12)
m = Mage("Gandalf", 12)
r = Rogue("Shades", 11)

attack(r, m)
attack(m, r)
attack(k, m)

attack(r, m)
attack(m, k)
attack(k, m)  # Gandalf dies
attack(r, m)  # Shades is attacking a dead character

while r.is_alive() and k.is_alive():
    attack(r, k)
    if k.is_alive():
        attack(k, r)

winner = r.get_name() if r.is_alive() else k.get_name()
print("{} wins the battle!".format(winner))  # Arthur

Shades (Rogue, 11, 550/550) attacks Gandalf (Mage, 12, 600/600) and hits for 147 damage. Gandalf still has 453 (75.5%) health
Gandalf (Mage, 12, 453/600) attacks Shades (Rogue, 11, 550/550) and hits for 136 damage. Shades still has 414 (75.3%) health
Arthur (Knight, 12, 600/600) attacks Gandalf (Mage, 12, 453/600) and hits for 129 damage. Gandalf still has 324 (54.0%) health
Shades (Rogue, 11, 414/550) attacks Gandalf (Mage, 12, 324/600) and hits for 147 damage. Gandalf still has 177 (29.5%) health
Gandalf (Mage, 12, 177/600) attacks Arthur (Knight, 12, 600/600) and hits for 101 damage. Arthur still has 499 (83.2%) health
Arthur (Knight, 12, 499/600) attacks Gandalf (Mage, 12, 177/600) and hits for 129 damage. Gandalf still has 48 (8.0%) health
Shades (Rogue, 11, 414/550) attacks Gandalf (Mage, 12, 48/600) and hits for 48 damage. Gandalf died!
Shades (Rogue, 11, 414/550) attacks Arthur (Knight, 12, 499/600) and hits for 74 damage. Arthur still has 425 (70.8%) health
Arthur (Knight, 12,

In [140]:
from unittest import TestCase


class Knight(Character):
    def __init__(self, *kwargs):
        super(Knight, self).__init__(*kwargs)
        self.__reduce_dmg = (1 - 0.25)
        self.__reduce_atk = (1 - 0.2)

    def _get_caused_dmg(self, other):
        assert isinstance(other, Character)
        assert self is not other
        damage = int(round((self._lvl*10+(self._lvl - other._lvl))*0.8))
        return max(1, damage)

    def _take_physical_damage(self, amount):
        assert isinstance(amount, int)
        assert amount >= 0
        self._health_cur = max(0, self._health_cur -
                               int(round(amount * self.__reduce_dmg)))


class TestBattle(TestCase):

    def test_basic_attack(self):
        sut = Knight("Arthur", 3)
        actual = sut.get_health()
        expected = (150, 150)
        self.assertEqual(expected, actual)

    def test_knight_attack(self):
        k = Knight("Arthur", 3)
        r = Rogue("Shades", 3)
        k.attack(r)
        actual = r.get_health()[0]
        expected = 150 - round(0.8 * (3 * 10 + 0))
        self.assertEqual(expected, actual)

In [142]:
a = TestBattle()
a.test_basic_attack()
a.test_knight_attack()

In [144]:
class TestCharacter(TestCase):

    def test_getters(self):
        sut = Character("C", 1)
        self.assertEqual("C", sut.get_name())
        self.assertEqual(1, sut.get_lvl())
        self.assertEqual((50, 50), sut.get_health())

    def test_repr(self):
        actual = repr(Character("C", 1))
        expected = "C (Character, 1, 50/50)"
        self.assertEqual(expected, actual)

    def test_invalid_name(self):
        with self.assertRaises(AssertionError):
            Character("", 1)

    def test_invalid_level(self):
        with self.assertRaises(AssertionError):
            Character("C", 0)

    def test_correct_health_scaling(self):
        sut = Character("C", 2)
        self.assertEqual((100, 100), sut.get_health())

    def test_take_physical_damage(self):
        sut = MockCharacter("C", 2)
        sut.physical_dmg(13)
        self.assertEqual((87, 100), sut.get_health())

    def test_take_magical_damage(self):
        sut = MockCharacter("C", 2)
        sut.magical_dmg(27)
        self.assertEqual((73, 100), sut.get_health())

    def test_attack_is_delegated(self):
        sut = Character("C", 1)
        other = MockCharacter("C", 1)
        sut.attack(other)
        self.assertEqual([10], other.physical_taken)

    def test_attacker_does_not_get_dmg(self):
        sut = MockCharacter("C", 1)
        sut.attack(Character("D", 1))
        self.assertEqual([], sut.physical_taken)

    def test_dmg_is_correct(self):
        sut = MockCharacter("C", 1)
        actual = sut.get_caused_dmg(Character("o", 1))
        expected = 10
        self.assertEqual(expected, actual)

    def test_dmg_scaling(self):
        sut = MockCharacter("C", 2)
        actual = sut.get_caused_dmg(Character("o", 2))
        expected = 20
        self.assertEqual(expected, actual)

    def test_dmg_lvl_diff_neg(self):
        sut = MockCharacter("C", 3)
        actual = sut.get_caused_dmg(Character("o", 4))
        expected = 29
        self.assertEqual(expected, actual)

    def test_dmg_lvl_diff_pos(self):
        sut = MockCharacter("C", 4)
        actual = sut.get_caused_dmg(Character("o", 2))
        expected = 42
        self.assertEqual(expected, actual)

    def test_dmg_lvl_huge_diff(self):
        sut = MockCharacter("C", 1)
        actual = sut.get_caused_dmg(Character("o", 100))
        expected = 1
        self.assertEqual(expected, actual)

    def test_taken_physical_dmg_must_be_int(self):
        with self.assertRaises(AssertionError):
            MockCharacter("C", 1).physical_dmg(12.3)

    def test_taken_physical_dmg_must_be_positive(self):
        with self.assertRaises(AssertionError):
            MockCharacter("C", 1).physical_dmg(-1)

    def test_taken_magical_dmg_must_be_int(self):
        with self.assertRaises(AssertionError):
            MockCharacter("C", 1).magical_dmg(12.3)

    def test_taken_magical_dmg_must_be_positive(self):
        with self.assertRaises(AssertionError):
            MockCharacter("C", 1).magical_dmg(-1)

    def test_no_negative_health_physical(self):
        sut = MockCharacter("C", 1)
        sut.physical_dmg(100)
        self.assertEqual((0, 50), sut.get_health())

    def test_no_negative_health_magical(self):
        sut = MockCharacter("C", 1)
        sut.magical_dmg(100)
        self.assertEqual((0, 50), sut.get_health())

    def test_is_alive(self):
        sut = MockCharacter("C", 1)
        self.assertTrue(sut.is_alive())
        sut.physical_dmg(100)
        self.assertFalse(sut.is_alive())

    def test_dead_chars_cannot_attack(self):
        sut = MockCharacter("C", 1)
        sut.physical_dmg(100)
        with self.assertRaises(Warning):
            sut.attack(Character("B", 2))

    def test_chars_cannot_attack_themselves(self):
        sut = Character("C", 1)
        with self.assertRaises(AssertionError):
            sut.attack(sut)

    def test_chars_cannot_attack_non_characters(self):
        sut = Character("C", 1)
        with self.assertRaises(AssertionError):
            sut.attack(1)


class MockCharacter(Character):
    """This mock class is used to expose protected methods for testing and
    also to capture the damage taken to simplify test assertions."""

    def __init__(self, name, lvl):
        super().__init__(name, lvl)
        self.physical_taken = []
        self.magical_taken = []

    def get_caused_dmg(self, other):
        return self._get_caused_dmg(other)

    def _take_physical_damage(self, amount):
        self.physical_taken.append(amount)
        super()._take_physical_damage(amount)

    def _take_magical_damage(self, amount):
        self.magical_taken.append(amount)
        super()._take_magical_damage(amount)

    def physical_dmg(self, amount):
        self._take_physical_damage(amount)

    def magical_dmg(self, amount):
        self._take_magical_damage(amount)

In [145]:
tc = TestCharacter()

In [158]:
tc.test_getters()
tc.test_repr()
tc.test_invalid_name()
tc.test_invalid_level()
tc.test_correct_health_scaling()
tc.test_take_physical_damage()
tc.test_take_magical_damage()

tc.test_attacker_does_not_get_dmg()
tc.test_dmg_lvl_huge_diff()
tc.test_taken_physical_dmg_must_be_int()
tc.test_taken_physical_dmg_must_be_positive()
tc.test_taken_magical_dmg_must_be_int()
tc.test_taken_magical_dmg_must_be_positive()
tc.test_no_negative_health_physical()
tc.test_no_negative_health_magical()
tc.test_is_alive()

tc.test_chars_cannot_attack_themselves()
tc.test_chars_cannot_attack_non_characters()

In [ ]:
# tc.test_attack_is_delegated()
# tc.test_dmg_is_correct()
# tc.test_dmg_scaling()
# tc.test_dmg_lvl_diff_neg()
# tc.test_dmg_lvl_diff_pos()
# tc.test_dead_chars_cannot_attack()